### Question1

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from tqdm import tqdm 

class ImageNetMiniDataset(Dataset):
    def __init__(self, txt_file, transform=None):
        self.image_labels = []
        with open(txt_file, 'r') as file:
            lines = file.readlines()
            for line in lines:
                path, label = line.strip().split()
                self.image_labels.append((path, int(label)))
        self.transform = transform

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, idx):
        path = "./images"
        img_path, label = self.image_labels[idx]

        img_path = os.path.join(path,img_path)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [12]:
from torchvision import transforms

# 定義數據轉換
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

validation_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# 加載數據集
train_dataset = ImageNetMiniDataset('images/train.txt', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

validation_dataset = ImageNetMiniDataset('images/val.txt', transform=validation_transform)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)

test_dataset = ImageNetMiniDataset('images/test.txt', transform=validation_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class DynamicConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(DynamicConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.dynamic_gen = nn.Linear(in_channels, out_channels * in_channels * kernel_size * kernel_size)
        
    def forward(self, x):
        b, c, h, w = x.size()
        
        # 生成動態卷積核
        dynamic_weights = self.dynamic_gen(x.mean(dim=[2, 3]))  # [b, out_channels * in_channels * kernel_size * kernel_size]
        dynamic_weights = dynamic_weights.view(b * self.out_channels, self.in_channels, self.kernel_size, self.kernel_size)
        
        # 初始化輸出
        output = torch.zeros(b, self.out_channels, h, w).to(x.device)
        
        # 應用動態卷積
        for i in range(b):
            weight = dynamic_weights[i*self.out_channels:(i+1)*self.out_channels]  # [out_channels, in_channels, kernel_size, kernel_size]
            input_i = x[i].unsqueeze(0)  # [1, in_channels, h, w]
            output[i] = F.conv2d(input_i, weight, padding=self.kernel_size//2)
        
        return output

class SimpleModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(SimpleModel, self).__init__()
        self.dynamic_conv = DynamicConv(in_channels, 64, kernel_size=3)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = self.dynamic_conv(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定義模型
model = SimpleModel(in_channels=3, num_classes=50)  # 假設有50個分類
model = model.to("cuda")

In [14]:
import torch.optim as optim

# 超參數設置
learning_rate = 0.001
epochs = 10

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 訓練和驗證循環
for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to("cuda")
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.to("cuda"))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')
    
    # 驗證階段
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.to("cuda")

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted.cpu().detach() == labels.cpu().detach()).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy}%')

# 最終測試
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.cpu().detach() == labels.cpu().detach()).sum().item()
test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy}%')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1, Loss: 3.791535262387208


 10%|█         | 1/10 [07:11<1:04:47, 431.93s/it]

Validation Accuracy: 5.555555555555555%
Epoch 2, Loss: 3.7322031782131004


 20%|██        | 2/10 [14:16<57:00, 427.55s/it]  

...

 90%|█████████ | 9/10 [1:02:24<06:55, 415.06s/it]

Validation Accuracy: 6.0%
Epoch 10, Loss: 3.7172854808845903


100%|██████████| 10/10 [1:09:15<00:00, 415.54s/it]

Validation Accuracy: 5.333333333333333%


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)

### Question 2

In [1]:
import torch
import torch.nn as nn

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = x.mean(dim=[2, 3])
        y = self.relu(self.fc1(y))
        y = self.sigmoid(self.fc2(y))
        y = y.view(b, c, 1, 1)
        return x * y

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.se = SEBlock(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [3]:
class ResNetSE(nn.Module):
    def __init__(self, num_classes=100):
        super(ResNetSE, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64, 2)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        layers = []
        layers.append(BasicBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 定义数据集
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

validation_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageNetMiniDataset('images/train.txt', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

validation_dataset = ImageNetMiniDataset('images/val.txt', transform=validation_transform)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)

test_dataset = ImageNetMiniDataset('images/test.txt', transform=validation_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义模型
model = ResNetSE(num_classes=50)  # 假设有50个分类

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练和验证循环
for epoch in tqdm(range(10)):  # 训练10个epoch
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')
    
    # 验证阶段
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy}%')

# 最终测试
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy}%')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1, Loss: 3.1491384749460702


 10%|█         | 1/10 [41:25<6:12:50, 2485.59s/it]

Validation Accuracy: 21.555555555555557%
Epoch 2, Loss: 2.5542553910101304


 20%|██        | 2/10 [1:21:04<5:23:02, 2422.77s/it]

...

 90%|█████████ | 9/10 [5:45:44<37:53, 2273.60s/it]  

Validation Accuracy: 51.77777777777778%
Epoch 10, Loss: 1.2646733518802757


100%|██████████| 10/10 [6:23:34<00:00, 2301.42s/it]

Validation Accuracy: 56.44444444444444%


Test Accuracy: 58.22222222222222%
